In [1]:
from random import choice, choices
import base64
from Crypto.Cipher import AES

BLOCKSIZE = 16

def Pad(s: bytes):
    assert type(s) is bytes
    npad = BLOCKSIZE - len(s) % BLOCKSIZE
    return s + bytes( [npad] * npad )

def Unpad(s: bytes):
    assert type(s) is bytes and len(s) % BLOCKSIZE == 0
    last = s[-1]
    assert 1 <= last <= 16 and all( byte == last for byte in s[-last:] )
    return s[:-last]

with open('strings.txt', mode = 'rb') as file:
    strings = [ base64.decodebytes(line) for line in file ]

def Chunkerize(x, chunksize, strict = True):
    x = list(x)
    assert len(x) % chunksize == 0 if strict else True
    for n in range( len(x) // chunksize ):
        yield x[ n*chunksize : (n+1)*chunksize ]

In [2]:
def PadPlaintext(plaintext: bytes, blocksize = 16):
    npad = blocksize - len(plaintext) % blocksize
    return plaintext + bytes([npad]) * npad

def XOR(X: bytes, Y: bytes) -> bytes:
    assert type(X) is bytes and type(Y) is bytes and len(X) == len(Y)
    return bytes([ x^y for x, y in zip(X, Y) ])

def EncryptCBC(plaintext, key, initialization):
    BLOCKSIZE = 16
    plaintext = PadPlaintext(plaintext, BLOCKSIZE)
    assert type(initialization) is bytes and len(initialization) == BLOCKSIZE
    ECBcipher = AES.new(key, AES.MODE_ECB)
    
    plain_blocks = [ bytes(block) for block in Chunkerize(plaintext, BLOCKSIZE) ]
    cipher_blocks = [None] * len(plain_blocks)
    
    for n in range(len(plain_blocks)):
        if n == 0:
            plain_block = XOR(plain_blocks[n], initialization)
            cipher_blocks[n] = ECBcipher.encrypt(plain_block)
        else:
            plain_block = XOR(plain_blocks[n], cipher_blocks[n-1])
            cipher_blocks[n] = ECBcipher.encrypt(plain_block)
    
    return b''.join(cipher_blocks)

def DecryptCBC(ciphertext, key, initialization):
    BLOCKSIZE = 16
    assert len(ciphertext) % BLOCKSIZE == 0
    assert type(initialization) is bytes and len(initialization) == BLOCKSIZE
    ECBcipher = AES.new(key, AES.MODE_ECB)
    
    cipher_blocks = [ bytes(block) for block in Chunkerize(ciphertext, BLOCKSIZE) ]
    plain_blocks = [None] * len(cipher_blocks)
    
    for n in range(len(plain_blocks)):
        if n == 0:
            plain_block = ECBcipher.decrypt(cipher_blocks[n])
            plain_blocks[n] = XOR(plain_block, initialization)
        else:
            plain_block = ECBcipher.decrypt(cipher_blocks[n])
            plain_blocks[n] = XOR(plain_block, cipher_blocks[n-1])
            
    plaintext = b''.join(plain_blocks)
    return plaintext
 

In [3]:
key = b'\x13c\xb1I\x17\xe7dj\xd5?\xbe\xb9f\x03\xa1\x84'
strings = [ base64.decodebytes(line) for line in open('strings.txt', mode = 'rb') ]
for string in strings:
    print(string, '| len =', len(string))

def Function1():    
    initialization = bytes( choices(range(256), k = 16) )
    
    plaintext = choice(strings)
    ciphertext = EncryptCBC(plaintext, key, initialization)
    return initialization, ciphertext

def Function2(initialization, ciphertext):
    try:
        plaintext = DecryptCBC(ciphertext, key, initialization)
        plaintext = Unpad(plaintext)
        return True
    except:
        return False

b'000000Now that the party is jumping' | len = 35
b"000001With the bass kicked in and the Vega's are pumpin'" | len = 56
b'000002Quick to the point, to the point, no faking' | len = 49
b"000003Cooking MC's like a pound of bacon" | len = 40
b"000004Burning 'em, if you ain't quick and nimble" | len = 48
b'000005I go crazy when I hear a cymbal' | len = 37
b'000006And a high hat with a souped up tempo' | len = 43
b"000007I'm on a roll, it's time to go solo" | len = 41
b"000008ollin' in my five point oh" | len = 32
b'000009ith my rag-top down so my hair can blow' | len = 45


In [4]:
# Start with a ciphertext I know to start easy

plaintext = b'A'*(BLOCKSIZE+1)
initialization = b'iiiiiiiiiiiiiiii'
ciphertext = EncryptCBC(plaintext, key, initialization)

In [5]:
DecryptCBC(ciphertext, key, initialization)

b'AAAAAAAAAAAAAAAAA\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f'

In [6]:
# Solve the last char to get a feel

new_ciphertext = list(ciphertext)

for char in range(256):
    if char == ciphertext[15]:
        continue
    new_ciphertext[15] = char
    if Function2(initialization, bytes(new_ciphertext)):
        print(char^1^ciphertext[15])

15


In [7]:
# Ok so last char is 15
# let's do second to last

last_char = 15

new_ciphertext = list(ciphertext)

new_ciphertext[15] = 2 ^ last_char ^ ciphertext[15]
new_ciphertext[14] = 2 ^ 15 ^ ciphertext[14]

DecryptCBC(bytes(new_ciphertext), key, initialization)
Function2(initialization, bytes(new_ciphertext[:]))

True

In [8]:
# Now solve in general for two blocks

def SolveBlockPair(block1, block2):
    assert type(block1) is type(block2) is bytes
    assert len(block1) == len(block2) == BLOCKSIZE
    
    # Find the last block
    pad = 1
    possible_solutions = list()
    new_ciphertext = list(block1)
    for char in range(0xFF): 
        new_ciphertext[BLOCKSIZE-1] = char
        if Function2(bytes(new_ciphertext), block2):
            possible_solutions += [ char ]
    
    # In the event that multiple solutions have been found... just pick one randomly and go for it!
    # This isn't necessarily the smartest choice... but it will get the right answer eventually
    known_chars = [ pad^block1[BLOCKSIZE-1]^choice(possible_solutions) ]
    
    # Now work on the others
    for pad in range(2, BLOCKSIZE+1):
        assert len(known_chars) == pad-1
        idx = BLOCKSIZE-pad # index of char we are currently trying to solve for
        new_ciphertext = list(block1)
        assert len(known_chars) == len(block1[idx+1:])
        new_ciphertext[idx+1:] = [ pad^known_char^cipher_char for known_char, cipher_char in zip(known_chars, block1[idx+1:]) ]
        possible_solutions = list()
        
        for char in range(0xFF):
            new_ciphertext[idx] = char
            decrypted = DecryptCBC(block2, key, bytes(new_ciphertext)) # For debugging, delete later
            assert len(decrypted) == BLOCKSIZE
            assert all( item == pad for item in decrypted[idx+1:] )
            if Function2(bytes(new_ciphertext), block2):
                possible_solutions += [ char ]
        
        char = choice(possible_solutions)
        known_chars = [ char^pad^block1[idx], *known_chars ]

    return known_chars
    


In [9]:
# Now solve in general for arbitary length

def Attack(initialization, ciphertext):
    assert len(initialization) == BLOCKSIZE
    assert len(ciphertext) % BLOCKSIZE == 0
    blocks = [ bytes(chunk) for chunk in Chunkerize(initialization+ciphertext, BLOCKSIZE) ]
    
    return b''.join( bytes(SolveBlockPair(block1, block2)) for block1, block2 in zip(blocks[:-1], blocks[1:]) )

In [11]:
Attack(initialization, ciphertext)

b'AAAAAAAAAAAAAAAAA\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f'

In [28]:
initialization, ciphertext = Function1()
print(initialization)
print(ciphertext)

b'{2\xe9\xc9\xdbq3=D\xfe4J\x9b\xbb\xf2-'
b'h\nj|:\xbd\x1a\xfb\xf1\xda\xf2j\xb9\xd3\xadh\xefVt\xea\\\x85\xeb\xee\x8c*\x9eo\xe3N\x10\xcc\xfb\x02p{_!\x91\x07[p\x05\xa5t\xca\xb7V8E?\xe5\x91\xb3\xdbRt}\x1d\xd5\x14\x11\xbd\x08'


In [29]:
for n in range(100):
    try:
        print(Attack(initialization, ciphertext))
        break
    except:
        pass

b'000002Quick to the point, to the point, no faking\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f'


In [ ]:
# It seems like for certain randomly generated initializations the problem cannot be solved by my code...
# I believe this seems to occur most frequently with a pad value of BLOCKSIZE
# I'm sure I could track this down... but I'm goint to call this problem mostly solved and move on to the next one